# Модель
Мне не удалось добится хороших результатов используя данную модель, но тем не менее, идея мне показалась довольно интересной.
### Основная идея
1. Построить разбиение множества языков, в соответствии, например, со схожестью алфавитов.
2. Для каждой группы языков построить классификатор, который хорошо умеет различать языки в рамках данной группы.
3. Построить классификатор, который бы в качестве ответа для текста выдавал группу языков. Далее мы бы пользовались классификатором для выбранной группы.

### Подготовка данных (для группы языков)
Сначала строится алфавит, представляющий из себя объединение алфавитов языков в группе.

Для каждого текста выполняются следующие преобразования:

1. Из текста удаляются не alphanum символы (включая пробелы).
2. Текст переводится в нижний регистр.
3. Текст обрезается до определенной длины.
4. Для каждого символа в тексте выполняется one-hot-encoding, таким образом из текста мы получаем вектор, состоящий из векторов для символов из текста.

### Классификатор (для группы языков)
Классификатор представляет из себя RNN с LSTM слоями.

# Эксперименты
Возникли следующие проблемы при попытке разбить множество языков на группы:
1. В data-train есть ошибки, то есть некоторым языкам поставлен в соответствие текст на другом языке. Это приводит к тому, что алфавиты разных групп языков сильно пересекаются.
2. Количество текстов в разных группах языков может очень сильно отличаться.

В виду данных проблем, была предпринята попытка построить классификатор для группы языков, в которую входят все языки.

Возможно, если потратить больше времени на обучение классификатора, то можно было бы добиться хороших результатов.


In [1]:
import nltk


lines = [line.rstrip('\n') for line in open('data-train.txt')]

def text_filter(text):
    return ''.join([c.lower() for c in text if c.isalpha()])

l = {}
l_cnt = {}

for i in range(1, len(lines)):
    lang, text = lines[i].split('\t', 1)
    
    text = text_filter(text)
    
    f = l_cnt.get(lang, 0)
    l_cnt[lang] = f + 1
    
    if (l.get(lang) is None):
        l[lang] = [text]
    else:
        l[lang].append(text) 


In [5]:
min_len = int(1e5)

for land in l:
    for sent in l[lang]:
        min_len = min(min_len, len(sent))
        
print(min_len)

132


In [7]:
for lang in l:
    for i in range(len(l[lang])):
        l[lang][i] = l[lang][i][:min_len] 

In [45]:
lang_groups = [
    ['ru', 'uk', 'be', 'bg', 'mk'],
    ['en', 'de', 'fr', 'it', 'la', 'nl', 'pt', 'sv', 'es'],
    ['sr', 'sl', 'sk', 'pl', 'cs', 'ro', 'hr', 'hsb', 'lt', 'lv'],
    ['hy'],
    ['el']
]

lang_ids = [
    {'ru':0, 'uk':1, 'be':2, 'bg':3, 'mk':4},
    {'en':0, 'de':1, 'fr':2, 'it':3, 'la':4, 'nl':5, 'pt':6, 'sv':7, 'es':8},
    {'sr':0, 'sl':1, 'sk':2, 'pl':3, 'cs':4, 'ro':5, 'hr':6, 'hsb':7, 'lt':8, 'lv':9},
    {'hy':0},
    {'el':0}
]

l_arr = ['be','bg','cs','de','el','en','es','fr','hr','hsb','hy','it','la','lt','lv','mk','nl','pl','pt','ro','ru','sk','sl','sr','sv','uk']

l_ids = {}
l_i = 0
l_from_id = {}

for lang in l_arr:
    l_ids[lang] = l_i
    l_from_id[l_i] = lang
    l_i += 1

In [13]:
def alphabet_of_lang_group(group):
    a = set([])
    for lang in group:
        for sent in l[lang]:
            for c in sent:
                a.add(c)
    alphabet = {}
    i = 0
    for c in a:
        alphabet[c] = i
        i += 1
    return alphabet

In [14]:
alphabets = []
for group in lang_groups:
    alphabets.append(alphabet_of_lang_group(group))

In [22]:
A = alphabet_of_lang_group(l_arr)

print(len(A))

215


In [23]:
print({ lang: len(l[lang]) for lang in l_arr })

{'ru': 173608, 'cs': 1799, 'la': 500, 'it': 3395, 'sk': 1604, 'es': 775, 'sv': 341, 'hr': 1757, 'fr': 9454, 'pt': 355, 'hsb': 182, 'nl': 2175, 'ro': 228, 'mk': 1811, 'sr': 1722, 'hy': 4533, 'sl': 1624, 'en': 72351, 'be': 15023, 'lv': 2448, 'el': 227, 'bg': 15043, 'uk': 33901, 'lt': 120, 'pl': 24663, 'de': 17708}


In [34]:
def intersection(i, j):
    inter = set(alphabets[i].keys()).intersection(set(alphabets[j].keys()))
    print(inter, len(inter), len(alphabets[i]), len(alphabets[j]))
    
intersection(0,1)

{'б', 'x', 'я', 'f', 'х', 'b', 'ž', 'ï', 'н', 'ц', 'v', 'з', 'î', 'ж', 'ê', 'а', 'у', 'ç', 'j', 'ь', 'q', 'ъ', 'ü', 'э', 'd', 'ß', 'á', 'p', 'в', 'њ', 'ч', 'ј', 'w', 'д', 'i', 'п', 'u', 'œ', 'и', 'a', 'l', 'ю', 'c', 'o', 'k', 'к', 'ù', 'm', 'г', 'y', 'й', 'h', 'щ', 'ó', 'о', 'ô', 'ö', 'л', 'ы', 'û', 's', 'р', 'é', 't', 'â', 'ш', 'z', 'n', 'e', 'r', 'g', 'č', 'è', 'с', 'ф', 'е', 'м', 'ä', 'т', 'à'} 80 123 92


In [39]:
import random
from random import shuffle


def train_data_for_lang_group(group):
    data = []
    for lang in group:
        for sent in l[lang]:
            data.append((sent, lang))
    shuffle(data)
    return data

In [40]:
train_data = train_data_for_lang_group(l_arr)

In [42]:
print(len(train_data))

387347


In [43]:
def data_chunk(data, begin, length=10000):
    return data[begin:begin+length-1]

In [73]:
import numpy as np


def sent2vec(sent, alphabet):
    vec = []
    for c in sent:
        v = [0] * len(alphabet)
        v[alphabet[c]] = 1
        vec.append(np.array(v))
    return np.array(vec)

def lang2vec(lang, group_lang_ids):
    v = [0] * len(group_lang_ids)
    v[group_lang_ids[lang]] = 1
    return np.array(v)

def prepare_data(data, alphabet, group_lang_ids):
    x = []
    y = []
    for sent, lang in data:
        if len(sent) == min_len:
            x.append(sent2vec(sent, alphabet))
            y.append(lang2vec(lang, group_lang_ids))
    return np.array(x), np.array(y)

In [47]:
chunk = data_chunk(train_data, 0)

x_train, y_train = prepare_data(chunk, A, l_ids)

In [50]:
print(x_train.shape, x_train[0].shape)

(9999, 132, 215) (132, 215)


In [54]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution2D, Flatten, Reshape
from keras.optimizers import SGD
from keras.layers import Masking, Dense, Dropout, Activation
from keras.layers import LSTM, SimpleRNN, GRU


def compile_model(model, learning_rate=0.01, d=1e-6):
    sgd = SGD(lr=learning_rate, decay=d) 
    model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['categorical_accuracy'])
    
    return model

def train_model(model, train_dataset, train_labels, bs=80, epoch=20):
    model.fit(train_dataset, train_labels, batch_size=bs, nb_epoch=epoch)

def rnn_model(alphabet_len, group_len):
    model = Sequential([
#         Masking(mask_value=0., input_shape=(seq_len, vec_len)),
        LSTM(alphabet_len, input_shape=(132, alphabet_len), return_sequences=True),
#         LSTM(50, return_sequences=True),
        LSTM(50),
        Dense(group_len, activation='softmax')
    ])
    return model

In [55]:
model = compile_model(rnn_model(len(A), len(l_arr)))

In [ ]:
chunk = data_chunk(train_data, 10000)

x_train, y_train = prepare_data(chunk, A, l_ids)

In [72]:
for i in range(len(x_train)):
    if len(x_train[i]) != 132:
        print (x_train[i], i, len(x_train[i]))

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]] 6060 51


In [ ]:
train_model(model, x_train, y_train)